In [1]:
# Import required packages
import matplotlib as mpl
from matplotlib import colors

%matplotlib inline
mpl.rcParams['figure.figsize'] = (15, 10) #this increases the inline figure size to 15 tall x 10 wide

import seaborn
import pandas as pd
import geopandas as gpd
import pysal
import numpy as np
import mapclassify
import matplotlib.pyplot as plt
import pylab as pl
import adjustText as aT
import matplotlib as mtp

import warnings
warnings.filterwarnings('ignore') # Change settings so that warnings are not displayed

import contextily as cx
from shapely.geometry import Polygon
from shapely.geometry import Point
import plotly.express as px
from pysal.explore import esda
from pysal.lib import weights
from splot.esda import plot_moran
from splot.esda import moran_scatterplot
from splot.esda import plot_local_autocorrelation
from splot.esda import lisa_cluster
from esda.moran import Moran_Local

# Loading a few new packages
from scipy import stats
from pysal.model import spreg
import statsmodels.formula.api as sm

/opt/anaconda3/envs/NCG613_Project/lib/python3.8/site-packages/pysal/model/spvcm/abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [2]:
# Importing data
hp = gpd.read_file('hp.geojson') #2016 housing points with some additional distance factors calculated
lb = gpd.read_file('london_boroughs.geojson')
oa = gpd.read_file('OutputAreas.geojson')
sch = gpd.read_file('London_Schools.geojson')

In [3]:
# Setting CRS to British National Grid
hp.crs = {'init': 'epsg:27700'}
lb.crs = {'init': 'epsg:27700'}
oa.crs = {'init': 'epsg:27700'}
sch.crs = {'init': 'epsg:27700'}

In [4]:
# Renaming output areas borough name
oa = oa.rename(columns = {'NAME_2': 'NAME'}, inplace = False)

In [5]:
# Using numpy, creating three new binary columns based on whether the houses are detached, flats, or freehold
hp['Detached'] = np.where(hp['propertytype']=='D', 1, 0)
hp['Flats'] = np.where(hp['propertytype']=='F', 1, 0)
hp['Freehold'] = np.where(hp['duration']=='F', 1, 0)

In [6]:
# Joining house price data to output area data using Geopandas - saved in new dataset hp2
hp2 = gpd.sjoin(hp,oa)
pd.set_option('display.max_columns', None)
hp2.head()

,ID,east,north,price,propertytype,duration,fl_area,No_Rooms,Price_Per_sqm,Postcode,logprice,log_fl_area,Dist_School,Dist_AcConv,Dist_FSM,Dist_Priv,Dist_Of1,Dist_Of12,Dist_Prim12,Dist_Sec12,Dist_VAS,Dist_AcSponsor,geometry,Detached,Flats,Freehold,index_right,geo_code,label,NAME,POP,POPDENS,HH,HECTARES,DEPRHH,EMPRES,STUDENT,PROFSCI,Trees,TreeDens
0,1,528925,157964,995000,D,F,288.0,7,3454.861111,CR5 3EZ,5.997823,2.459392,943.612208,1852.404114,1852.404114,943.612208,1972.702968,1303.989264,1303.989264,2733.169772,1303.989264,2733.169772,POINT (528925.000 157964.000),1,0,1,19031,E00005097,E92000001E09000008E00005097,Croydon,399.0,6.621308,133.0,60.26,0.375940,197,0.062914,0.127451,21.0,0.348490
16174,16175,529109,157935,720000,D,F,141.4,7,5091.937765,CR5 3EH,5.857332,2.150449,784.413794,1705.065981,1705.065981,784.413794,1795.243716,1291.652043,1291.652043,2547.375905,1291.652043,2547.375905,POINT (529109.000 157935.000),1,0,1,19031,E00005097,E92000001E09000008E00005097,Croydon,399.0,6.621308,133.0,60.26,0.375940,197,0.062914,0.127451,21.0,0.348490
17146,17147,529166,157845,535000,T,F,143.0,6,3741.258741,CR5 3EQ,5.728354,2.155336,681.558508,1705.221686,1705.221686,681.558508,1756.834938,1373.847881,1373.847881,2472.446764,1373.847881,2472.446764,POINT (529166.000 157845.000),0,0,1,19031,E00005097,E92000001E09000008E00005097,Croydon,399.0,6.621308,133.0,60.26,0.375940,197,0.062914,0.127451,21.0,0.348490
1,2,533634,180992,1300000,F,L,140.0,3,9285.714286,EC3N 1NT,6.113943,2.146128,249.353163,1398.845953,543.945769,479.688441,249.353163,249.353163,249.353163,1133.884033,249.353163,2112.389405,POINT (533634.000 180992.000),0,1,0,12112,E00000029,E92000001E41000324E00000029,City of London,225.0,14.696280,127.0,15.31,0.740157,125,0.095000,0.136364,110.0,7.184847
29,30,533634,180992,680000,F,L,84.0,3,8095.238095,EC3N 1NT,5.832509,1.924279,249.353163,1398.845953,543.945769,479.688441,249.353163,249.353163,249.353163,1133.884033,249.353163,2112.389405,POINT (533634.000 180992.000),0,1,0,12112,E00000029,E92000001E41000324E00000029,City of London,225.0,14.696280,127.0,15.31,0.740157,125,0.095000,0.136364,110.0,7.184847


In [7]:
# Setting variable names from columns of interest - including deprivation factor DEPRHH
variable_names = ['log_fl_area', 'No_Rooms', 'Flats', 'Detached', 'DEPRHH']

In [8]:
# Calculating log versions of all distances
hp2['Log_Dist_School'] = np.log(hp2['Dist_School']+1)
hp2['Log_Dist_AcSponsor'] = np.log(hp2['Dist_AcSponsor']+1)
hp2['Log_Dist_FSM'] = np.log(hp2['Dist_FSM']+1)
hp2['Log_Dist_Priv'] = np.log(hp2['Dist_Priv']+1)
hp2['Log_Dist_Of1'] = np.log(hp2['Dist_Of1']+1)
hp2['Log_Dist_Of12'] = np.log(hp2['Dist_Of12']+1)
hp2['Log_Dist_Prim12'] = np.log(hp2['Dist_Prim12']+1)
hp2['Log_Dist_Sec12'] = np.log(hp2['Dist_Sec12']+1)
hp2['Log_Dist_VAS'] = np.log(hp2['Dist_VAS']+1)

In [9]:
# Additional in additional school distance parameter
# This is done one at a time so the effect of each type of school on the base model can be observed
base_names = variable_names
dist_names = variable_names + ['Log_Dist_School']

# + ['Log_Dist_School']
# + ['Log_Dist_AcSponsor']
# + ['Log_Dist_FSM']
# + ['Log_Dist_Priv']
# + ['Log_Dist_Of1']
# + ['Log_Dist_Of12']
# + ['Log_Dist_Prim12']
# + ['Log_Dist_Sec12']
# + ['Log_Dist_VAS']

In [10]:
# Creating a distance-based spatial weights matrix and generating spatial lags of the residuals

w = weights.DistanceBand.from_dataframe(hp2, 500) # Weights based on features within 500 meters

## Spatial Lag Model (SAR)

In [11]:
m5 = spreg.GM_Lag(hp2[['logprice']].values, hp2[dist_names].values,
                     w=w, name_y='logprice', name_x=dist_names, robust='white')
print(m5.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    logprice                Number of Observations:       70925
Mean dependent var  :      5.6704                Number of Variables   :           8
S.D. dependent var  :      0.2493                Degrees of Freedom    :       70917
Pseudo R-squared    :      0.6279
Spatial Pseudo R-squared:  0.6211

White Standard Errors
------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       4.3688496       0.0151697     287.9990857       0.0000000
         log_fl_area       0.8580283       0.0080298     106.8550708       0.0000000
            No_Rooms       0.0067125 

## Spatial Error Model (SEM)

In [12]:
m6 = spreg.GM_Error_Het(hp2[['logprice']].values, hp2[dist_names].values,
                     w=w, name_y='logprice', name_x=dist_names)
print(m6.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES (HET)
---------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    logprice                Number of Observations:       70925
Mean dependent var  :      5.6704                Number of Variables   :           7
S.D. dependent var  :      0.2493                Degrees of Freedom    :       70918
Pseudo R-squared    :      0.5996
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       4.5631034       0.0154689     294.9854182       0.0000000
         log_fl_area       0.8654076       0.0082895     104.3978864       0.00

## Spatial Lag + Error Model (SARMA)

In [13]:
m7 = spreg.GM_Combo_Het(hp2[['logprice']].values, hp2[dist_names].values,
                     w=w, name_y='logprice', name_x=dist_names)
print(m7.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED TWO STAGE LEAST SQUARES (HET)
-------------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    logprice                Number of Observations:       70925
Mean dependent var  :      5.6704                Number of Variables   :           8
S.D. dependent var  :      0.2493                Degrees of Freedom    :       70917
Pseudo R-squared    :      0.6279
Spatial Pseudo R-squared:  0.6211
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       4.3688496       0.0151697     287.9990857       0.0000000
         log_fl_area     

## Spatial Regimes Models

In [14]:
# Camden

hpcam = hp2[(hp2["NAME"]=="Camden")]
wcam = weights.DistanceBand.from_dataframe(hpcam, 500)
m7cam = spreg.GM_Error_Het(hpcam[['logprice']].values, hpcam[dist_names].values,
                     w=wcam, name_y='logprice', name_x=dist_names)
print(m7cam.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES (HET)
---------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    logprice                Number of Observations:        1315
Mean dependent var  :      5.9246                Number of Variables   :           7
S.D. dependent var  :      0.2912                Degrees of Freedom    :        1308
Pseudo R-squared    :      0.8254
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       4.5248546       0.0714447      63.3336276       0.0000000
         log_fl_area       0.7741897       0.0324542      23.8548173       0.00

In [15]:
# Barking and Dagenham

hpbd = hp2[(hp2["NAME"]=="Barking and Dagenham")]
wbd = weights.DistanceBand.from_dataframe(hpbd, 500)
m7bd = spreg.GM_Error_Het(hpbd[['logprice']].values, hpbd[dist_names].values,
                     w=wbd, name_y='logprice', name_x=dist_names)
print(m7bd.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES (HET)
---------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    logprice                Number of Observations:        1718
Mean dependent var  :      5.4417                Number of Variables   :           7
S.D. dependent var  :      0.1305                Degrees of Freedom    :        1711
Pseudo R-squared    :      0.6804
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       4.8299149       0.0511821      94.3673382       0.0000000
         log_fl_area       0.3574165       0.0285508      12.5186085       0.00

In [16]:
# Kensington and Chelsea

hpken = hp2[(hp2["NAME"]=="Kensington and Chelsea")]
wken = weights.DistanceBand.from_dataframe(hpken, 500)
m7ken = spreg.GM_Error_Het(hpken[['logprice']].values, hpken[dist_names].values,
                     w=wken, name_y='logprice', name_x=dist_names)
print(m7ken.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES (HET)
---------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    logprice                Number of Observations:        1182
Mean dependent var  :      6.1308                Number of Variables   :           7
S.D. dependent var  :      0.3638                Degrees of Freedom    :        1175
Pseudo R-squared    :      0.8185
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       4.4427054       0.0794388      55.9261687       0.0000000
         log_fl_area       0.9930694       0.0358418      27.7070022       0.00

In [17]:
# Hackney

hphack = hp2[(hp2["NAME"]=="Hackney")]
whack = weights.DistanceBand.from_dataframe(hphack, 500)
m7hack = spreg.GM_Error_Het(hphack[['logprice']].values, hphack[dist_names].values,
                     w=whack, name_y='logprice', name_x=dist_names)
print(m7hack.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIALLY WEIGHTED LEAST SQUARES (HET)
---------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :    logprice                Number of Observations:        1390
Mean dependent var  :      5.7244                Number of Variables   :           7
S.D. dependent var  :      0.2220                Degrees of Freedom    :        1383
Pseudo R-squared    :      0.6690
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       4.7750497       0.0671424      71.1182608       0.0000000
         log_fl_area       0.6708238       0.0346094      19.3827268       0.00